In [4]:
import requests
import pandas as pd
from plyer import notification
from datetime import datetime
import sqlite3

In [5]:
def extracao(url,base):
    data = str(datetime.now())
    req = requests.get(url)
    status_code = req.status_code
    data_json = req.json()
    if status_code == 200:
        df = pd.DataFrame(data_json)
        message = 'Requisição realizada com sucesso'
        notification.notify(
        title=f'ATENÇÃO: {message}'
        , message = message
        , app_name='alerta'
        , timeout=10)

        return(df)
        
    elif status_code >= 400:
        message = 'Erro na requisição'
    elif status_code == 200:
        message = 'Requisição concluída com sucesso'
    else:
        print ('Outros tipos de erro')
    
    notification.notify(
        title=f'ATENÇÃO: {message}'
        , message = str('Carregamento da base %s. \n %s' % (base,data))
        , app_name=''
        , timeout=10)

In [6]:
base_municipios = extracao('https://brasilapi.com.br/api/ibge/municipios/v1/sp?providers=dados-abertos-br,gov,wikipedia','base_municipios')

In [4]:
base_municipios.head()

,nome,codigo_ibge
0,ADAMANTINA,3500105
1,ADOLFO,3500204
2,AGUAI,3500303
3,AGUAS DA PRATA,3500402
4,AGUAS DE LINDOIA,3500501


In [5]:
base_uf = extracao('https://brasilapi.com.br/api/ibge/uf/v1','base_uf')

In [6]:
base_uf.head()

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


In [7]:
base_estado_sp = extracao('https://brasilapi.com.br/api/ibge/uf/v1/sp','base_estado_sp')

In [8]:
base_estado_sp.head()

,id,sigla,nome,regiao
id,35,SP,São Paulo,3
nome,35,SP,São Paulo,Sudeste
sigla,35,SP,São Paulo,SE


In [9]:
base_pix = extracao('https://brasilapi.com.br/api/pix/v1/participants','base_pix')

In [10]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z


Tratamento

In [11]:
base_uf.head()

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


In [12]:
sigla_regiao = [sr['sigla'] for sr in base_uf['regiao']]
nome_regiao = [nr['nome'] for nr in base_uf['regiao']]
sigla_estado = [estado for estado in base_uf['sigla']]
nome_estado = [estado for estado in base_uf['nome']]

In [13]:
base_uf_tratada = pd.DataFrame({'Sigla': sigla_estado, 'Nome': nome_estado, 'Sigla Região': sigla_regiao, 'Nome Região': nome_regiao})

In [14]:
base_uf_tratada.head()

,Sigla,Nome,Sigla Região,Nome Região
0,RO,Rondônia,N,Norte
1,AC,Acre,N,Norte
2,AM,Amazonas,N,Norte
3,RR,Roraima,N,Norte
4,PA,Pará,N,Norte


In [15]:
base_estado_sp.head()

,id,sigla,nome,regiao
id,35,SP,São Paulo,3
nome,35,SP,São Paulo,Sudeste
sigla,35,SP,São Paulo,SE


In [16]:
base_estado_sp_tratada = base_estado_sp.reset_index()

In [17]:
base_estado_sp_tratada

,index,id,sigla,nome,regiao
0,id,35,SP,São Paulo,3
1,nome,35,SP,São Paulo,Sudeste
2,sigla,35,SP,São Paulo,SE


In [18]:
base_estado_sp_tratada1 = base_estado_sp_tratada[base_estado_sp_tratada['index'] == 'nome']

In [19]:
base_estado_sp_tratada1

,index,id,sigla,nome,regiao
1,nome,35,SP,São Paulo,Sudeste


In [20]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z


In [21]:
from datetime import datetime

In [22]:
base_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ispb                     798 non-null    object
 1   nome                     798 non-null    object
 2   nome_reduzido            798 non-null    object
 3   modalidade_participacao  798 non-null    object
 4   tipo_participacao        798 non-null    object
 5   inicio_operacao          798 non-null    object
dtypes: object(6)
memory usage: 37.5+ KB


In [23]:
base_pix['inicio_operacao'] = pd.to_datetime(base_pix['inicio_operacao'])

In [24]:
base_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   ispb                     798 non-null    object             
 1   nome                     798 non-null    object             
 2   nome_reduzido            798 non-null    object             
 3   modalidade_participacao  798 non-null    object             
 4   tipo_participacao        798 non-null    object             
 5   inicio_operacao          798 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 37.5+ KB


In [25]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03 09:30:00+00:00
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03 09:30:00+00:00
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03 09:30:00+00:00
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03 09:30:00+00:00
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03 09:30:00+00:00


In [26]:
base_pix['inicio_operacao'] = base_pix['inicio_operacao'].dt.strftime("%d/%m/%Y")

In [27]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,03/11/2020
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,03/11/2020
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,03/11/2020
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,03/11/2020
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,03/11/2020


Salvar em banco de dados

In [35]:
conn = sqlite3.connect('base_uf_tratada.db')
base_uf.to_sql('base_uf_tratada', conn, if_exists='replace', index=False)
query = "SELECT * FROM base_uf_tratada"
df = pd.read_sql(query, conn)
print(df)

conn.close()

ProgrammingError: Error binding parameter 4: type 'dict' is not supported

In [36]:
base_uf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      27 non-null     int64 
 1   sigla   27 non-null     object
 2   nome    27 non-null     object
 3   regiao  27 non-null     object
dtypes: int64(1), object(3)
memory usage: 996.0+ bytes


In [32]:
conn = sqlite3.connect('base_municipios.db')
base_municipios.to_sql('base_municipios', conn, if_exists='replace', index=False)
query = "SELECT * FROM base_municipios"
df = pd.read_sql(query, conn)
print(df)

conn.close()

                 nome codigo_ibge
0          ADAMANTINA     3500105
1              ADOLFO     3500204
2               AGUAI     3500303
3      AGUAS DA PRATA     3500402
4    AGUAS DE LINDOIA     3500501
..                ...         ...
640        VOTORANTIM     3557006
641       VOTUPORANGA     3557105
642          ZACARIAS     3557154
643         CHAVANTES     3557204
644      ESTIVA GERBI     3557303

[645 rows x 2 columns]


In [33]:
conn = sqlite3.connect('base_estado_sp_tratada1.db')
base_estado_sp.to_sql('base_estado_sp_tratada1', conn, if_exists='replace', index=False)
query = "SELECT * FROM base_estado_sp_tratada1"
df = pd.read_sql(query, conn)
print(df)

conn.close()

   id sigla       nome   regiao
0  35    SP  São Paulo        3
1  35    SP  São Paulo  Sudeste
2  35    SP  São Paulo       SE


In [34]:
conn = sqlite3.connect('base_pix.db')
base_pix.to_sql('base_pix', conn, if_exists='replace', index=False)
query = "SELECT * FROM base_pix"
df = pd.read_sql(query, conn)
print(df)

conn.close()

         ispb                                               nome  \
0    00000000                               BANCO DO BRASIL S.A.   
1    00000208                       BRB - BANCO DE BRASILIA S.A.   
2    00068987  COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...   
3    00075847  COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...   
4    00106180  CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...   
..        ...                                                ...   
793  95213211  COOPERATIVA DE CRéDITO, POUPANçA E INVESTIMENT...   
794  95424891  COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...   
795  95594941  COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...   
796  97259253  COOPERATIVA E ECONOMIA E CRÉDITO MÚTUO DOS MÉD...   
797  97489280  COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...   

                                         nome_reduzido  \
0                                   BCO DO BRASIL S.A.   
1                           BRB - BCO DE BRASILIA S.A.   
2        